# Initialization

In [1]:
import os
from pathlib import Path
import signal
import time

# PyBoolNet initialization
from pyboolnet.repository import get_all_names, get_bnet
from pyboolnet.file_exchange import bnet2primes
from pyboolnet.trap_spaces import compute_trap_spaces

# BioLQM init
import biolqm

# trappist init
import trappist

# timeout with signals
class TimeOutException(Exception):
   pass

def handler(_signnum, _frame):
    print("Timeout")
    raise TimeOutException()
    
signal.signal(signal.SIGALRM, handler)

This notebook has been executed using the docker image `colomoto/colomoto-docker:2022-05-01`

<Handlers.SIG_DFL: 0>

In [2]:
# compute trapspaces using PyBoolNet directly
def bench_pyboolnet(models, max=1000, timeout=120):
    for model in models:
        print(model)
        signal.alarm(timeout)
        try:
            start = time.perf_counter()
            primes = bnet2primes(model + ".bnet")
            # end = time.perf_counter()
            # print(f"implicants: {end - start}s")
            tspaces = compute_trap_spaces(primes, "min", max_output=max)
            end = time.perf_counter()
            print(f"{len(tspaces)} trap spaces {end - start:.2f}\n===")
        except TimeOutException:
            print("DNF\n===")
        signal.alarm(0)

In [3]:
# compute trapspaces using trappist
def bench_trappist(models, max=1000, timeout=120):
    for model in models:
        print(model)
        signal.alarm(timeout)
        try:
            start = time.perf_counter()
            biolqm.save(biolqm.load(model + ".bnet"), model + ".pnml")
            # end = time.perf_counter()
            # print(f"conversion: {end - start}s")
            tspaces = trappist.compute_trap_spaces(model + ".pnml", max_output=max)
            end = time.perf_counter()
            print(f"{len(list(tspaces))} trap spaces {end - start:.2f}\n===")
        # we need to catch our TimeOutException, but it raises a Py4J exception too…
        except:
            print("DNF\n===")
        signal.alarm(0)

## PyBoolNet repository

Unfortunately all models are very easy

In [4]:
# all models from the PyBoolNet repository https://github.com/hklarner/pyboolnet/tree/master/pyboolnet/repository
models = get_all_names()

# only the slowest ones
# models = ['jaoude_thdiff', 'n12c5', 'remy_tumorigenesis', 'selvaggio_emt']

# save .bnet files locally
for model in models:
    net = get_bnet(model)
    with open(model + ".bnet", "wt") as f:
        print(net, file=f)

In [5]:
bench_pyboolnet(models)

arellano_rootstem
4 trap spaces 0.06
===
calzone_cellfate
27 trap spaces 0.02
===
dahlhaus_neuroplastoma
32 trap spaces 0.04
===
davidich_yeast
12 trap spaces 0.03
===
dinwoodie_life
7 trap spaces 0.02
===
dinwoodie_stomatal
1 trap spaces 0.02
===
faure_cellcycle
2 trap spaces 0.03
===
grieco_mapk
18 trap spaces 0.04
===
irons_yeast
1 trap spaces 0.04
===
jaoude_thdiff
INFO there are possibly more than 1000 trap spaces.
INFO increase MaxOutput to find out.
1000 trap spaces 1.50
===
klamt_tcr
8 trap spaces 0.03
===
krumsiek_myeloid
6 trap spaces 0.02
===
multivalued
4 trap spaces 0.02
===
n12c5
5 trap spaces 30.96
===
n3s1c1a
2 trap spaces 0.02
===
n3s1c1b
2 trap spaces 0.02
===
n5s3
3 trap spaces 0.02
===
n6s1c2
3 trap spaces 0.03
===
n7s3
3 trap spaces 0.02
===
raf
2 trap spaces 0.02
===
randomnet_n15k3
3 trap spaces 0.02
===
randomnet_n7k3
10 trap spaces 0.02
===
remy_tumorigenesis
25 trap spaces 1.73
===
saadatpour_guardcell
1 trap spaces 0.02
===
selvaggio_emt
INFO there are possib

In [6]:
bench_trappist(models)

arellano_rootstem
4 trap spaces 0.28
===
calzone_cellfate
27 trap spaces 0.05
===
dahlhaus_neuroplastoma
32 trap spaces 0.04
===
davidich_yeast
12 trap spaces 0.02
===
dinwoodie_life
7 trap spaces 0.02
===
dinwoodie_stomatal
1 trap spaces 0.02
===
faure_cellcycle
2 trap spaces 0.02
===
grieco_mapk
18 trap spaces 0.07
===
irons_yeast
1 trap spaces 0.03
===
jaoude_thdiff
1000 trap spaces 0.21
===
klamt_tcr
8 trap spaces 0.03
===
krumsiek_myeloid
6 trap spaces 0.02
===
multivalued
4 trap spaces 0.02
===
n12c5
5 trap spaces 0.03
===
n3s1c1a
2 trap spaces 0.01
===
n3s1c1b
2 trap spaces 0.02
===
n5s3
3 trap spaces 0.02
===
n6s1c2
3 trap spaces 0.02
===
n7s3
3 trap spaces 0.02
===
raf
2 trap spaces 0.02
===
randomnet_n15k3
3 trap spaces 0.02
===
randomnet_n7k3
10 trap spaces 0.02
===
remy_tumorigenesis
25 trap spaces 0.04
===
saadatpour_guardcell
1 trap spaces 0.01
===
selvaggio_emt
1000 trap spaces 0.19
===
tournier_apoptosis
3 trap spaces 0.02
===
xiao_wnt5a
4 trap spaces 0.02
===
zhang_tlg

## Local set of big models

Collected from various bibliographic sources (see article for details)

In [7]:
# all models from the trappist's benchmark directory
models = [str(Path().absolute() / "models" / name[:-5]) for name in os.listdir("models") if name.endswith(".bnet")]

In [8]:
bench_pyboolnet(models)

/notebook/models/CASCADE3
1 trap spaces 82.86
===
/notebook/models/KYNURENINE-PATHWAY
INFO there are possibly more than 1000 trap spaces.
INFO increase MaxOutput to find out.
1000 trap spaces 74.71
===
/notebook/models/SN5
Timeout
DNF
===
/notebook/models/ERBB-RECEPTOR-SIGNALING
Timeout
DNF
===
/notebook/models/Executable_file_for_cholocystokinin_model
INFO there are possibly more than 1000 trap spaces.
INFO increase MaxOutput to find out.
1000 trap spaces 1.55
===
/notebook/models/korkut_2015a
Timeout
DNF
===
/notebook/models/SIGNALING-PATHWAY-FOR-BUTANOL-PRODUCTION
INFO there are possibly more than 1000 trap spaces.
INFO increase MaxOutput to find out.
1000 trap spaces 0.43
===
/notebook/models/Corral_ThIL17diff_15jan2021
Timeout
DNF
===
/notebook/models/HIV-1
Timeout
DNF
===
/notebook/models/Executable_file_for_CaSQ_derived_mast_cell_activation_model
INFO there are possibly more than 1000 trap spaces.
INFO increase MaxOutput to find out.
1000 trap spaces 0.49
===
/notebook/models/Co

In [9]:
bench_trappist(models)

/notebook/models/CASCADE3
1 trap spaces 12.83
===
/notebook/models/KYNURENINE-PATHWAY
1000 trap spaces 4.22
===
/notebook/models/SN5
Timeout
DNF
===
/notebook/models/ERBB-RECEPTOR-SIGNALING
1000 trap spaces 9.27
===
/notebook/models/Executable_file_for_cholocystokinin_model
1000 trap spaces 3.57
===
/notebook/models/korkut_2015a
1000 trap spaces 3.06
===
/notebook/models/SIGNALING-PATHWAY-FOR-BUTANOL-PRODUCTION
1000 trap spaces 0.66
===
/notebook/models/Corral_ThIL17diff_15jan2021
1000 trap spaces 0.91
===
/notebook/models/HIV-1
1000 trap spaces 3.25
===
/notebook/models/Executable_file_for_CaSQ_derived_mast_cell_activation_model
1000 trap spaces 0.56
===
/notebook/models/Colitis_associated_colon_cancer
10 trap spaces 0.22
===
/notebook/models/INTERFERON-1
1000 trap spaces 1.05
===
/notebook/models/TLGLSurvival
318 trap spaces 0.48
===
/notebook/models/cho_2016_all_pos
1000 trap spaces 3.00
===
/notebook/models/turei_2016
DNF
===
/notebook/models/Human_network
Timeout
DNF
===
/notebook